# Data Transformation Overview

## Project: Amazon Product Data Scraping

### Description
This document outlines the data transformation steps applied to the Amazon product data scraped using the `aiohttp` library for asynchronous requests and `BeautifulSoup` for HTML parsing. The purpose of these transformations is to clean and standardize the data for further analysis and use.

### Data Sources
- **Source:** Amazon
- **Data Fields:** Price, Model Name, Brand Name, Rating, Reviews, etc.
- **Date of Extraction:** 9/18/2024

### Transformation Steps
1. **Model Name Cleaning**
   - Process applied to standardize model names for brands such as Oppo, Nokia, Realme, and others.

2. **Price Formatting**
   - Adjustments made to clean and format price values, including handling currency codes and removing unwanted characters.

3. **Memory and Storage Extraction**
   - Extraction and standardization of RAM and storage capacities from product titles.

4. **Additional Data Processing**
   - Any other relevant transformations or cleaning steps applied to the dataset.

### Purpose
The transformations aim to ensure data consistency and accuracy, facilitating easier analysis and integration into data pipelines.


For detailed transformation logic and code snippets, refer to the sections below.

---

## Data Loading and Initial Inspection

### Importing Libraries
```python
import pandas as pd
import numpy as np
from datetime import datetime
import re

In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime 
import re
# Load the CSV file 
df = pd.read_csv("E:\\Optimized-Amazon-project\\phones-row-data\\RowData.csv")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 93 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   date_column                          254 non-null    object 
 1   product_url                          254 non-null    object 
 2   site                                 254 non-null    object 
 3   category                             254 non-null    object 
 4   Title                                239 non-null    object 
 5   Rate                                 216 non-null    float64
 6   Price                                190 non-null    object 
 7   Price Before Discount                52 non-null     object 
 8   Discount                             61 non-null     object 
 9   Image URL                            239 non-null    object 
 10  Description                          237 non-null    object 
 11  Brand Name                      

In [3]:
df.columns

Index(['date_column', 'product_url', 'site', 'category', 'Title', 'Rate',
       'Price', 'Price Before Discount', 'Discount', 'Image URL',
       'Description', 'Brand Name', 'Operating System', 'RAM Memory Installed',
       'CPU Model', 'CPU Speed', 'Memory Storage Capacity', 'Screen Size',
       'Resolution', 'Refresh Rate', 'Model Name', 'OS', 'Package Dimensions',
       'Item model number', 'Wireless communication technologies',
       'Connectivity technologies', 'GPS', 'Special features',
       'Other display features', 'Human interface input', 'Scanner Resolution',
       'Other camera features', 'Form factor', 'Color', 'Battery Power Rating',
       'Whats in the box', 'Item Weight', 'ASIN', 'Customer Reviews',
       'Best Sellers Rank', 'Date First Available', 'reviews',
       'Wireless Provider', 'Cellular Technology', 'Connectivity Technology',
       'Batteries', 'Product Dimensions', 'Manufacturer', 'Brand', 'Series',
       'Form Factor', 'Standing screen display s

In [4]:
# Display rows, columns 
df.shape

(254, 93)

In [5]:
# Ensure that all the Data are objects
df.dtypes

date_column          object
product_url          object
site                 object
category             object
Title                object
                     ...   
Battery Capacity     object
Warranty Type        object
Number of Ports     float64
Unit Count           object
Display Type         object
Length: 93, dtype: object

## Filtering Phone-Related Data

In [6]:
# Drop rows where the 'Title' column has no values
df = df.dropna(subset=["Title"], how='all', axis=0)

# Define keywords related to phones that may appear in 'Title' or 'Description'
phone_keywords = ['phone', 'smartphone', 'g', 'gb',]

# Filter out rows that are likely not phones based on the absence of phone-related keywords
non_phone_df = df[~(
    df['Title'].str.lower().str.contains('|'.join(phone_keywords), case=False, na=False) |
    df['Description'].str.lower().str.contains('|'.join(phone_keywords), case=False, na=False)
)]

# Display the non_phone_df DataFrame "For Debugging"
# non_phone_df

# Keep rows that are likely phones based on the presence of phone-related keywords
filtered_df = df[
    df['Title'].str.lower().str.contains('|'.join(phone_keywords), case=False, na=False) |
    df['Description'].str.lower().str.contains('|'.join(phone_keywords), case=False, na=False)
]

# Drop columns where all values are NaN
filtered_df = filtered_df.dropna(axis=1, how='all')

# Display the number of columns after filtering
filtered_df.shape

(238, 89)

## Data Cleaning and Debugging

### Debugging Checks Before and After Processing

To debug and inspect specific columns before and after filling missing values:

```python
# Debugging specific row and column values
filtered_df.loc[178, ['Operating System', 'OS']]
filtered_df['Brand Name'].count()
filtered_df['Connectivity technologies'].count()
filtered_df['Wireless Provider'].count()
filtered_df['RAM Memory Installed'].count()
filtered_df['Memory Storage Capacity'].count()


In [7]:
# Fill NaN values in 'column' with values from 'same column with different name'
filtered_df.loc[:,'Operating System'] = filtered_df['Operating System'].fillna(filtered_df['OS'])
filtered_df.loc[:,'Connectivity technologies'] = filtered_df['Connectivity technologies'].fillna(filtered_df['Connectivity Technology'])
filtered_df.loc[:,'Screen Size'] = filtered_df['Screen Size'].fillna(filtered_df['Standing screen display size'])
filtered_df.loc[:,'Wireless Provider'] = filtered_df['Wireless Provider'].fillna(filtered_df['Wireless communication technologies'])
filtered_df.loc[:,'Other display features'] = filtered_df['Other display features'].fillna(filtered_df['Display resolution'])

# Define a function to fill in 'Brand Name'
def fill_brand_name(row):
    if pd.notna(row['Brand Name']):
        return row['Brand Name']
    elif pd.notna(row['Manufacturer']):
        return row['Manufacturer']
    else:
        return row['Title'].split()[0]  # Take the first word of the title

# Apply the function to each row in the DataFrame
filtered_df.loc[:,'Brand Name'] = filtered_df.apply(fill_brand_name, axis=1)

# Function to extract RAM and Storage
def extract_memory_capacity(row):
    title = row['Title'].lower()  
    description = row['Description'].lower() if pd.notna(row['Description']) else "" # Handle Nan
    
    text = title + " " + description
    
    # Find all 'number GB/gb' in the title
    matches = re.findall(r'(\d+)\s*(GB|gb)', text)
    
    # Initialize variables for RAM and storage
    ram_capacity = None
    storage_capacity = None
    
    for match in matches:
        value, unit = match  # Get the value and unit
        
        # Construct the full capacity string
        capacity_str = f"{value}{unit}"
        
        # Check proximity context for RAM
        if 'ram' in title:
            ram_match = re.search(r'(\d+)\s*(GB|gb)\s*ram', title)
            if ram_match and f"{ram_match.group(1)}{ram_match.group(2)}" == capacity_str:
                ram_capacity = capacity_str
                
        # Check proximity context for Storage (storage/rom)
        if 'storage' in title or 'rom' in title:
            storage_match = re.search(r'(\d+)\s*(GB|gb)\s*(storage|rom)', title)
            if storage_match and f"{storage_match.group(1)}{storage_match.group(2)}" == capacity_str:
                storage_capacity = capacity_str
        
        # If no explicit mention of 'ram' or 'storage', make an educated guess
        if not ram_capacity and int(value) < 16:
            ram_capacity = capacity_str
        elif not storage_capacity and int(value) >= 16:
            storage_capacity = capacity_str
    
    # Update the 'storage' column if it's null and storage value is found
    if pd.isnull(row['Memory Storage Capacity']) and storage_capacity:
        row['Memory Storage Capacity'] = storage_capacity
    
    # Update the 'ram_gb' column if it's null and RAM value is found
    if pd.isnull(row['RAM Memory Installed']) and ram_capacity:
        row['RAM Memory Installed'] = ram_capacity
    
    return row

# Apply the function to rows with missing storage values
filtered_df = filtered_df.apply(extract_memory_capacity, axis=1)

# Display the updated DataFrame
filtered_df


,date_column,product_url,site,category,Title,Rate,Price,Price Before Discount,Discount,Image URL,...,Included Components,Model number,Manufacturer Part Number,Headphones Ear Placement,Headphones Form Factor,Noise Control,Sensitivity,Headphones Jack,Wireless Communication Technology,Display Type
0,2024-09-16,https://www.amazon.eg/-/en/Samsung-Storage-Tit...,amazon_egy,mobile phones,"Samsung Galaxy S24 Ultra, AI Phone, 256GB Stor...",4.1,"47,900.",NaN,NaN,https://m.media-amazon.com/images/I/719HV2e6-s...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-09-16,https://www.amazon.eg/-/en/New-Apple-iPhone-Pl...,amazon_egy,mobile phones,New Apple iPhone 14 Plus (128 GB) - Starlight,4.7,"34,990.","Was: EGP 35,990.00",-3%,https://m.media-amazon.com/images/I/51S9HOc4UL...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-09-16,https://www.amazon.eg/-/en/Xiaomi-Redmi-Green-...,amazon_egy,mobile phones,"Xiaomi Redmi Note 13 (Mint Green 8GB RAM, 256 ...",4.1,"8,650.","Was: EGP 8,990.00",-4%,https://m.media-amazon.com/images/I/51Q+-U8vAV...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-09-16,https://www.amazon.eg/-/en/HONOR-6936520833290...,amazon_egy,mobile phones,Honor X7b 6-256 GREEN,NaN,"8,690.",NaN,NaN,https://m.media-amazon.com/images/I/51ykLhmalQ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2024-09-16,https://www.amazon.eg/-/en/Redmi-Note-Midnight...,amazon_egy,mobile phones,"Redmi Note 13 Pro (Midnight Black 12GB RAM, 51...",4.0,"15,850.",NaN,NaN,https://m.media-amazon.com/images/I/51hrgXBsTa...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2024-09-16,https://www.amazon.eg/-/en/Nokia-android-smart...,amazon_egy,mobile phones,"Nokia c10 android smartphone, dual sim, 2gb ra...",3.3,"2,899.",NaN,NaN,https://m.media-amazon.com/images/I/81LrJ-Q-Tg...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,2024-09-16,https://www.amazon.eg/-/en/realme-Note-Factory...,amazon_egy,mobile phones,realme Note 50 DUAL SIM 64GB ROM + 3GB RAM (GS...,3.3,NaN,NaN,NaN,https://m.media-amazon.com/images/I/71v1qIODQq...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251,2024-09-16,https://www.amazon.eg/-/en/Samsung-Galaxy-Phan...,amazon_egy,mobile phones,"Samsung Galaxy S23 Ultra, 256GB, Phantom Black...",4.4,"38,450.",NaN,NaN,https://m.media-amazon.com/images/I/71goZuIha-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252,2024-09-16,https://www.amazon.eg/-/en/OPPO-A55-Dual-128GB...,amazon_egy,mobile phones,OPPO A55 Dual SIM 4GB RAM+128GB ROM (6.5 Inche...,4.0,"6,250.",NaN,NaN,https://m.media-amazon.com/images/I/619NisKmiu...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Fill Missing OS Based on Brand

In [8]:
# Define a dictionary for popular operating systems based on the brand
popular_os = {
    'realme': 'Android',
    'oppo': 'Android',
    'nokia': 'Android',  # Newer Nokia smartphones
    'samsung': 'Android',
    'xiaomi': 'Android'
}

# Function to fill missing OS based on brand
def fill_operating_system(row):
    if pd.isnull(row['Operating System']) and row['Brand Name'].lower() in popular_os:
        row['Operating System'] = popular_os[row['Brand Name'].lower()]  # Fill with popular OS
    return row

# Apply the function to the DataFrame
filtered_df = filtered_df.apply(fill_operating_system, axis=1)

# Check the result
filtered_df[['Operating System']].isna().value_counts()


Operating System
False               238
Name: count, dtype: int64

#### Drop Unnecessary Columns

In [9]:
# Drop unnecessary columns
columns_to_drop = ['OS', 'Connectivity Technology', 'Standing screen display size', 
                   'Wireless communication technologies', 'Display resolution']

# Get the range of columns between 'Country of origin' and 'Display Type'
start_series1 = filtered_df.columns.get_loc('Country of origin')
end_series1 = filtered_df.columns.get_loc('Display Type')


# Get the range of columns between 'Product Dimensions' and 'Number Of Lithium Ion Cells'
start_series2 = filtered_df.columns.get_loc('Product Dimensions')
end_series12 = filtered_df.columns.get_loc('Number Of Lithium Ion Cells')

# Add those columns to the drop list
columns_to_drop.extend(filtered_df.columns[start_series1:end_series1+1])
columns_to_drop.extend(filtered_df.columns[start_series2:end_series12+1])

# Drop the columns
filtered_df = filtered_df.drop(columns=columns_to_drop)

# Display the remaining columns
filtered_df.columns

Index(['date_column', 'product_url', 'site', 'category', 'Title', 'Rate',
       'Price', 'Price Before Discount', 'Discount', 'Image URL',
       'Description', 'Brand Name', 'Operating System', 'RAM Memory Installed',
       'CPU Model', 'CPU Speed', 'Memory Storage Capacity', 'Screen Size',
       'Resolution', 'Refresh Rate', 'Model Name', 'Package Dimensions',
       'Item model number', 'Connectivity technologies', 'GPS',
       'Special features', 'Other display features', 'Human interface input',
       'Scanner Resolution', 'Other camera features', 'Form factor', 'Color',
       'Battery Power Rating', 'Whats in the box', 'Item Weight', 'ASIN',
       'Customer Reviews', 'Best Sellers Rank', 'Date First Available',
       'reviews', 'Wireless Provider', 'Cellular Technology', 'Batteries',
       'RAM'],
      dtype='object')

### Fill Missing 'Model Name' Values

In [10]:
# Filter rows where 'Model Name' is NaN
filtered_model_df = filtered_df[filtered_df['Model Name'].isna()] 
filtered_model_df[['Title', 'Model Name', 'Brand Name']].head(15)

,Title,Model Name,Brand Name
9,Realme C65 4G 6GB RAM + 128GB – Gold,NaN,realme
13,OPPO A18 (64-4GB) - Glowing Black,NaN,OPPO
15,"Xiaomi Redmi Note 10 5G, Dual Sim, 64Gb, 4Gb R...",NaN,Xiaomi
16,New Apple iPhone 14 Plus (128 GB) - Blue,NaN,Apple
24,Nokia 105 Feature Phones Bundle 105 wireless F...,NaN,Nokia
26,"realme 12+ 5G Smartphone 12 + 512 GB with NFC,...",NaN,realme
48,Realme C63 (128-6GB) Blue,NaN,Realme
52,"Xiaomi Pad 6 (Champagne 8GB RAM, 256 Storage) ...",NaN,Xiaomi
67,"Realme 12 4G, 8GB Ram Mediatek Dimensity 6100+...",NaN,Realme
85,"Realme C61 Mobile Phone, Dual SIM, 6GB RAM, 25...",NaN,realme


In [11]:
# Function to clean the text by stripping spaces and converting to lower or upper case
def clean_text(df, columns):
    for col in columns:
        df.loc[:, col] = df[col].str.strip().str.lower()
    return df

# Generalized function to extract the model name using regex and specific word counts
def extract_model_name(title, regex_pattern=None, word_count=3, capitalize=True):
    if pd.isna(title):
        return None
    
    title = title.lower() if capitalize else title.upper()
    
    if regex_pattern:
        model_match = re.search(regex_pattern, title)
        if model_match:
            return model_match.group(0).title() if capitalize else model_match.group(0).upper()

    # Default case: take the first few words
    return ' '.join(title.split()[:word_count]).title() if capitalize else ' '.join(title.split()[:word_count]).upper()

# Define brand-specific model extraction logic
def get_extraction_rule(brand, title):
    title_lower = title.lower()

    if brand == 'oppo':
        return {'regex': None, 'word_count': 4 if 'reno' in title_lower else 2}
    elif brand == 'realme':
        return {'regex': None, 'word_count': 4 if 'pro' in title_lower else 3}
    elif brand == 'nokia':
        return {'regex': r'^(nokia\s*\w+|\w+)', 'word_count': 3}
    elif brand == 'redmi':
        return {'regex': r'redmi\s*\d+[a-z]*', 'word_count': 3}
    elif brand == 'samsung':
        return {'regex': r'(galaxy\s+\w+\s*\d*\s*(5g|4g)?)', 'word_count': 3}
    elif brand == 'honor':
        return {'regex': r'(HONOR [\w\s]+?)(\s*\d+GB|\s*(?:4G|5G|LTE)|$)', 'word_count': 3, 'capitalize': False}
    elif brand == 'xiaomi':
        return {'regex': None, 'word_count': 5 if 'redmi' in title_lower else 3}
    elif brand == 'apple':
        return {'regex': r'^([a-z\s]+?)(\d+)\s*(\w*)\s*(pro|plus)?', 'word_count': 3}
    else:
        return {'regex': None, 'word_count': 3}

# Apply the model extraction rules based on the brand
def apply_model_extraction(row):
    brand = row['Brand Name']
    title = row['Title']
    
    if pd.notna(row['Model Name']):
        return row['Model Name']  # Keep existing model name if available

    rule = get_extraction_rule(brand, title)
    return extract_model_name(title, regex_pattern=rule.get('regex'), word_count=rule.get('word_count', 3), capitalize=rule.get('capitalize', True))

# Apply the cleaning function and the model extraction to the DataFrame
filtered_df = clean_text(filtered_df, ['Title', 'Brand Name'])
filtered_df.loc[:,'Model Name'] = filtered_df.apply(apply_model_extraction, axis=1)

# Display the results
filtered_df[['Title', 'Model Name', 'Brand Name']]

,Title,Model Name,Brand Name
0,"samsung galaxy s24 ultra, ai phone, 256gb stor...",Galaxy S24 Ultra,samsung
1,new apple iphone 14 plus (128 gb) - starlight,iPhone 14 Plus,apple
2,"xiaomi redmi note 13 (mint green 8gb ram, 256 ...",Redmi Note 13,xiaomi
4,honor x7b 6-256 green,HONOR X7b,honor
6,"redmi note 13 pro (midnight black 12gb ram, 51...",Redmi Note 13 Pro,xiaomi
...,...,...,...
249,"nokia c10 android smartphone, dual sim, 2gb ra...",Nokia C10,nokia
250,realme note 50 dual sim 64gb rom + 3gb ram (gs...,Note 50,realme
251,"samsung galaxy s23 ultra, 256gb, phantom black...",Samsung Galaxy S23,samsung
252,oppo a55 dual sim 4gb ram+128gb rom (6.5 inche...,Oppo A55,oppo


#### Convert Object Columns to Lowercase and Strip Whitespace

**Exclude**: 'Image URL' and 'product_url' from the conversion process to keep links correct.

In [12]:
# List of columns to exclude from conversion
exclude_columns = ['Image URL', 'product_url']

# Get object columns excluding the specified ones
object_columns = filtered_df.select_dtypes(include=['object']).columns
columns_to_clean = [col for col in object_columns if col not in exclude_columns]

# Converting all applicable object columns to lowercase and stripping whitespace
for col in columns_to_clean:
    filtered_df.loc[:, col] = filtered_df[col].str.lower().str.strip()

filtered_df

,date_column,product_url,site,category,Title,Rate,Price,Price Before Discount,Discount,Image URL,...,Item Weight,ASIN,Customer Reviews,Best Sellers Rank,Date First Available,reviews,Wireless Provider,Cellular Technology,Batteries,RAM
0,2024-09-16,https://www.amazon.eg/-/en/Samsung-Storage-Tit...,amazon_egy,mobile phones,"samsung galaxy s24 ultra, ai phone, 256gb stor...",4.1,"47,900.",NaN,NaN,https://m.media-amazon.com/images/I/719HV2e6-s...,...,233 g,b0cqz22q7l,4.14.1 out of 5 stars706 ratings4.1 out of 5 s...,#522 in electronics (see top 100 in electronic...,17 january 2024,"[{'reviewer': 'george', 'rating': '5.0 ', 'dat...",cellular,NaN,NaN,NaN
1,2024-09-16,https://www.amazon.eg/-/en/New-Apple-iPhone-Pl...,amazon_egy,mobile phones,new apple iphone 14 plus (128 gb) - starlight,4.7,"34,990.","was: egp 35,990.00",-3%,https://m.media-amazon.com/images/I/51S9HOc4UL...,...,NaN,b0bdj59rrq,NaN,NaN,15 september 2022,"[{'reviewer': 'azizbek sharobidinov', 'rating'...",unlocked for all carriers,5g,1 lithium polymer batteries required. (included),NaN
2,2024-09-16,https://www.amazon.eg/-/en/Xiaomi-Redmi-Green-...,amazon_egy,mobile phones,"xiaomi redmi note 13 (mint green 8gb ram, 256 ...",4.1,"8,650.","was: egp 8,990.00",-4%,https://m.media-amazon.com/images/I/51Q+-U8vAV...,...,175 g,b0cqkddcvk,4.14.1 out of 5 stars164 ratings4.1 out of 5 s...,#599 in electronics (see top 100 in electronic...,20 january 2024,"[{'reviewer': 'sofianamr', 'rating': '5.0 ', '...",3,NaN,1 lithium polymer batteries required.,NaN
4,2024-09-16,https://www.amazon.eg/-/en/HONOR-6936520833290...,amazon_egy,mobile phones,honor x7b 6-256 green,NaN,"8,690.",NaN,NaN,https://m.media-amazon.com/images/I/51ykLhmalQ...,...,NaN,b0cvhdld4p,NaN,NaN,12 february 2024,[],china mobile,4g,NaN,NaN
6,2024-09-16,https://www.amazon.eg/-/en/Redmi-Note-Midnight...,amazon_egy,mobile phones,"redmi note 13 pro (midnight black 12gb ram, 51...",4.0,"15,850.",NaN,NaN,https://m.media-amazon.com/images/I/51hrgXBsTa...,...,510 g,b0cqkdbvt8,4.04.0 out of 5 stars38 ratings4.0 out of 5 stars,"#5,367 in electronics (see top 100 in electron...",6 march 2024,"[{'reviewer': 'carlos torres torres', 'rating'...",3,NaN,1 lithium polymer batteries required. (included),12 gb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2024-09-16,https://www.amazon.eg/-/en/Nokia-android-smart...,amazon_egy,mobile phones,"nokia c10 android smartphone, dual sim, 2gb ra...",3.3,"2,899.",NaN,NaN,https://m.media-amazon.com/images/I/81LrJ-Q-Tg...,...,NaN,b0b39ln2n6,NaN,NaN,6 june 2022,"[{'reviewer': 'مهندس .ممدوح فرج', 'rating': '5...",NaN,NaN,1 lithium ion batteries required. (included),NaN
250,2024-09-16,https://www.amazon.eg/-/en/realme-Note-Factory...,amazon_egy,mobile phones,realme note 50 dual sim 64gb rom + 3gb ram (gs...,3.3,NaN,NaN,NaN,https://m.media-amazon.com/images/I/71v1qIODQq...,...,370 g,b0d1gwyh9x,3.33.3 out of 5 stars9 ratings3.3 out of 5 stars,"#15,673 in electronics (see top 100 in electro...",16 may 2024,"[{'reviewer': 'mohamed fathy', 'rating': '1.0 ...",unlocked for all carriers,lte,1 lithium ion batteries required. (included),3 gb
251,2024-09-16,https://www.amazon.eg/-/en/Samsung-Galaxy-Phan...,amazon_egy,mobile phones,"samsung galaxy s23 ultra, 256gb, phantom black...",4.4,"38,450.",NaN,NaN,https://m.media-amazon.com/images/I/71goZuIha-...,...,233 g,b0bt9fdz8n,"4.44.4 out of 5 stars2,587 ratings4.4 out of 5...","#1,762 in electronics (see top 100 in electron...",20 february 2023,"[{'reviewer': 'randa a.', 'rating': '5.0 ', 'd...",cellular,NaN,1 lithium ion batteries required. (included),12 gb
252,2024-09-16,https://www.amazon.eg/-/en/OPPO-A55-Dual-128GB...,amazon_egy,mobile phones,oppo a55 dual sim 4gb ram+128gb rom (6.5 inche...,4.0,"6,250.",NaN,NaN,https://m.media-amazon.com/images/I/619NisKmiu...,...,NaN,NaN,NaN,NaN,NaN,"[{'reviewer': 'ايمان س.', 'rating': '5.0 ', 'd...",china mobile,NaN,NaN,NaN


#### Prepare Final DataFrame

In [13]:
# Select relevant columns
final_df = filtered_df[['date_column','site','category','Brand Name', 'Model Name', 'Title', 'Price', 'Operating System', 
                                 'RAM Memory Installed', 'Memory Storage Capacity', 'Screen Size', 
                                 'Resolution', 'Refresh Rate', 'CPU Speed', 'Connectivity technologies', 
                                 'CPU Model', 'Color', 'Wireless Provider', 
                                 'Cellular Technology', 'reviews', 'Rate', 'Discount','product_url','Image URL','ASIN','Batteries','Item model number']]

final_df = final_df.copy()

final_df.rename(columns={
    'Brand Name':'brand', 'Model Name':'model_name', 'Title':'product_title',
    'Price':'price_egp', 'Operating System':'os','RAM Memory Installed' : 'ram_gb',
    'Memory Storage Capacity':'storage', 'Screen Size':'screen_size_in',
    'Resolution':'resolution', 'Refresh Rate':'refresh_rate_hz', 'CPU Speed':'cpu_speed_ghz',
    'Connectivity technologies':'connectivity_technology','CPU Model':'cpu_model',
    'Color':'color','Wireless Provider':'wireless_carrier',
    'Cellular Technology':'cellular_technology', 'reviews':'all_reviews',
    'Rate':'rate', 'Discount':'discount','Image URL':'image_url','ASIN':'asin','Batteries':'batteries','Item model number':'model_number'
},inplace=True)

final_df.columns

Index(['date_column', 'site', 'category', 'brand', 'model_name',
       'product_title', 'price_egp', 'os', 'ram_gb', 'storage',
       'screen_size_in', 'resolution', 'refresh_rate_hz', 'cpu_speed_ghz',
       'connectivity_technology', 'cpu_model', 'color', 'wireless_carrier',
       'cellular_technology', 'all_reviews', 'rate', 'discount', 'product_url',
       'image_url', 'asin', 'batteries', 'model_number'],
      dtype='object')

#### Clean Column Values

In [14]:
def clean_col_val(df, col, remove_patterns=None, strip=True):
    # Remove the provided patterns
    if remove_patterns:
        for pattern in remove_patterns:
            df.loc[:,col] = df[col].str.replace(pattern, "", regex=True)
            
    # Optionally strip whitespace
    if strip:
        df.loc[:,col] = df[col].str.strip()

    return df

final_df = clean_col_val(final_df, "refresh_rate_hz", remove_patterns=["hz|hertz|ghz"])
final_df = clean_col_val(final_df, "discount", remove_patterns=["-|%"])

#### Clean Price Values

In [15]:
def clean_price(price):
    if pd.isna(price):
        return price  # If the price is NaN return as is
    
    # Define a list of unwanted characters and words to remove
    unwanted_chars = [',', '₹', '$', '£', '€', '₣', '¥']  
    unwanted_words = ['egp', 'usd', 'inr', 'eur', 'gbp', 'aud', 'cny']
    
    # Remove unwanted words and characters
    original_price = price  # original price for checking
    price = price.lower()
    for word in unwanted_words:
        price = re.sub(r'\b' + re.escape(word) + r'\b', '', price)
    for char in unwanted_chars:
        price = price.replace(char, '')
    
    # Remove trailing '.00' or similar patterns for prices with currency codes
    if any(word in original_price.lower() for word in unwanted_words):
        # Remove trailing .00 or .xx
        price = re.sub(r'(\d+)(?:,\d{3})*(?:\.\d{2})$', r'\1', price)
    else:
        # For other prices, remove only commas
        price = price.replace(',', '')
    
    # Ensure only numeric characters are kept
    price = re.sub(r'\D', '', price)  
    
    # Remove any leading or trailing whitespace
    price = price.strip()
    
    return price

# Apply the function to the 'price_egp' column
final_df['price_egp'] = final_df['price_egp'].apply(clean_price)


## Define Functions to Clean and Format Model Names

In [16]:
final_df['brand'].unique()

array(['samsung', 'apple', 'xiaomi', 'honor', 'realme', 'oppo', 'nokia',
       'itel', 'redmi', 'infinix'], dtype=object)

In [17]:
def format_samsung_model(model):
    model = model.lower() 

    # Extract special terms like FE, Ultra, Plus, 5G, 4G
    special_terms = re.findall(r'\b(fe|ultra|plus|5g|4g)\b', model, re.IGNORECASE)
    special_terms = list(dict.fromkeys(special_terms))  # Remove duplicates while maintaining order
    special_terms_str = ' '.join(special_terms).upper()

    # Remove unwanted words and characters
    model = re.sub(r'\b(samsung|galaxy)\b', '', model).strip()
    model = re.sub(r'\b(5g|4g)\b', '', model).strip()  # Remove 5G/4G here to prevent duplication

    # Extract core model name
    if 'note' in model:
        match = re.search(r'note\s*(\d*)\s*(\w*)', model)
        if match:
            note_num, note_suffix = match.groups()
            core_model = f"Note {note_num} {note_suffix}".strip().title()
    else:
        match = re.search(r'([a-z]+\s*\d+(?:\s*[a-z]+)?)', model)
        if match:
            core_model = match.group(1).strip().title()
        else:
            core_model = model.strip().title()

    # Remove special terms from core_model if they're already present
    for term in special_terms:
        core_model = re.sub(rf'\b{term}\b', '', core_model, flags=re.IGNORECASE).strip()

    # Combine core model and special terms
    result = f'{core_model} {special_terms_str}'.strip()

    return result.lower()

def format_realme_model(model):
    model = model.lower() 
    
    # Remove brand name 'realme'
    model = re.sub(r'\b(realme)\b', '', model).strip()
    
    # Replace '+' with 'plus'
    model = re.sub(r'\+', ' plus', model)
    
    # Remove extra descriptors like "dual-sim" and "dual"
    model = re.sub(r'\bdual[-\s]?sim\b', '', model)
    model = re.sub(r'\bdual\b', '', model)
    
    # Remove any extra descriptive information in parentheses or after commas
    model = re.sub(r'\(.*?\)', '', model).strip()
    model = re.sub(r',.*$', '', model).strip()
    
    # Handle hyphens between numbers and specs
    model = re.sub(r'(\d+)-(\d+)', r'\1 \2', model)
    
    # Extract the core model name, keeping '4G' or '5G' if present
    model = re.sub(r'\b4g\b', '4g', model)  
    model = re.sub(r'\b5g\b', '5g', model)  
    
    return model.strip()  

def format_xiaomi_brand(model):
    model = model.lower()
    
    # Ensure 'xiaomi' is removed, but not 'redmi'
    model = re.sub(r'\b(xiaomi|mi|xioami)\b', '', model).strip()
    
    # Handle hyphens between numbers and specs  (e.g., 12-128 becomes 12 128)
    model = re.sub(r'(\d+)-(\d+)', r'\1 \2', model)
    
    # Remove extra descriptors like "dual-sim" and "dual"
    model = re.sub(r'\bdual[-\s]?sim\b', '', model)
    model = re.sub(r'\bdual\b', '', model)
    
    # Replace '+' with 'plus'
    model = re.sub(r'\+', ' plus', model)
    
    # Remove colors (e.g., "midnight black", "shiny gold")
    model = re.sub(r'\b(?:midnight|black|blue|gold|silver|white|gray|green|red|orange|pink|purple|yellow)\b', '', model)
    
    # Remove memory specs (e.g., "8gb", "128gb")
    model = re.sub(r'\b\d+gb\b', '', model)
    
    # Remove any extra descriptive information in parentheses, after commas, or after hyphens
    model = re.sub(r'\(.*?\)', '', model).strip()
    model = re.sub(r',.*$', '', model).strip()
    model = re.sub(r'[\-\/].*$', '', model).strip()
    
    # Handle hyphens between numbers and specs (e.g., 12-128 becomes 12 128)
    model = re.sub(r'(\d+)-(\d+)', r'\1 \2', model)
    
    # Remove extra years (e.g., '2022') or other numeric descriptors after core model
    model = re.sub(r'\b\d{4}\b', '', model).strip()
    
    # Keep '4G', '5G', 'NE', etc. if present
    model = re.sub(r'\b4g\b', '4g', model)
    model = re.sub(r'\b5g\b', '5g', model)
    model = re.sub(r'\bne\b', 'ne', model)

    # Remove any remaining trailing/leading spaces and return cleaned model
    return model.strip()


def format_honor_model(model):
    model = model.lower().strip()
    
    # Remove the brand name 'honor'
    model = re.sub(r'\bhonor\b', '', model).strip()
    
    # Remove any extra descriptive information in parentheses, after commas, or after hyphens
    model = re.sub(r'\(.*?\)', '', model).strip()
    model = re.sub(r',.*$', '', model).strip()
    model = re.sub(r'[\-\/].*$', '', model).strip()
    
    # Replace '+' with 'plus'
    model = re.sub(r'\+', ' plus', model)
    
    # Handle 'pro' and 'plus'
    model = re.sub(r'\bpro\b', 'pro', model)
    model = re.sub(r'\bplus\b', 'plus', model)
    
    # Remove memory specs, colors, and other unnecessary information
    model = re.sub(r'\b\d+gb\b', '', model).strip()  # Remove '4gb', '8gb', etc.
    model = re.sub(r'\b\d+\+\d+\b', '', model).strip()  # Remove '12+512'
    model = re.sub(r'\b(?:titanium|silver|gold|black|blue|white|red|green|pink|grey|purple|yellow)\b', '', model).strip()  # Remove colors
    
    # Ensure 'plus' is included in the model name if present
    model = re.sub(r'\bplus\b', 'plus', model)
    
    # Final cleanup: remove extra spaces and return formatted model name
    model = re.sub(r'\s+', ' ', model)  # Replace multiple spaces with a single space
        
    return model.strip()


def format_apple_model(model):
    """ 
    Clean and format the model name for Apple products, including core model number and important descriptors.
    """
    model = model.lower().strip()
    
    # Remove the brand name 'apple' and 'iphone'
    model = re.sub(r'\bapple\b', '', model).strip()
    model = re.sub(r'\biphone\b', '', model).strip()
    
    # Replace hyphens with spaces
    model = re.sub(r'-', ' ', model)
    
    # Remove memory specs and additional descriptors
    model = re.sub(r'\b\d+gb\b', '', model)
    model = re.sub(r'\b(?:blue|starlight|facetime|gre|with|new)\b', '', model)
    
    # Remove extra descriptive information in parentheses or after hyphens
    model = re.sub(r'\(.*?\)', '', model).strip()
    model = re.sub(r',.*$', '', model).strip()
    
    # Split by spaces to extract core model name and descriptors
    parts = model.split()
    
    # Extract the core model number or name and important descriptors ('pro', 'max')
    core_model = []
    for part in parts:
        if re.match(r'^\d+', part):  # Check if part starts with a number
            core_model.append(part)
        elif part in ['pro', 'max', 'mini', 'plus']:  # Important descriptors
            core_model.append(part)
    
    return ' '.join(core_model).strip()

def format_oppo_model(model):
    model = model.lower()
    
    # Remove brand name 'oppo'
    model = re.sub(r'\b(oppo)\b', '', model).strip()

    # Remove any descriptive words (e.g., 'Android Smartphone', 'Dual SIM Mobile', colors, region versions)
    model = re.sub(r'(android|smartphone|dual sim|mobile|\.\.\.|glowing|black|blue|uae|version)', '', model).strip()

    # Remove numbers followed by 'gb' or 'ram' (e.g., '128gb', '4gb ram')
    model = re.sub(r'\d+\s*(gb|ram)', '', model).strip()

    # Extract the core model name and retain '4G' or '5G' if present
    match = re.search(r'(reno\s*\d+\s*\w*|a\d+\s*\w*|5g|4g)', model, re.IGNORECASE)

    # If there's a match, clean the model name
    if match:
        core_model = match.group(0).strip().title()

    # Check if 4G or 5G is present and append it if found
    special_terms = re.findall(r'(5g|4g)', model, re.IGNORECASE)
    if special_terms:
        core_model += ' ' + ' '.join(special_terms).upper()

    return core_model.strip().lower()

def format_nokia_model(model, product_title):
    model = model.lower()
    
    if model == 'nokia':
        product_title = product_title.lower()
        
        # Replace '+' with 'plus'
        product_title = re.sub(r'\+', ' plus', product_title)
        
        # Remove 'nokia' and split the title into words
        product_title = re.sub(r'\bnokia\b', '', product_title).strip()
        words = product_title.split()
        
        if len(words) == 0:
            return ''  # Return empty if no words are left after removing 'nokia'
        
        # Handle cases where we have fewer than 2 words remaining
        if len(words) == 1:
            return words[0]
        
        # Extract the second word and the third if it's '4G' or '5G'
        base_model = words[0] + ' ' + words[1]
        
        if len(words) > 2 and words[2] in ['4g', '5g']:
            base_model += ' ' + words[2]
        return base_model.strip().lower()
    else:
        # Replace '+' with 'plus'
        model = re.sub(r'\+', ' plus', model)
        # Remove 'nokia'
        model = re.sub(r'\bnokia\b', '', model).strip()
        return model.strip()


def format_infinix_model(model):
    model = model.lower()
    
    # Replace '+' with 'plus'
    model = re.sub(r'\+', ' plus', model)
    
    # Remove the brand name 'infinix' if present
    model = re.sub(r'\binfinix\b', '', model).strip()
    
    # Remove any extra descriptive information in parentheses or after commas
    model = re.sub(r'\(.*?\)', '', model).strip()
    model = re.sub(r',.*$', '', model).strip()
    
    # Handle cases with just the core model name
    model = model.strip().lower()
    
    return model

def format_redmi_model(model):
    """ 
    Ensure that the model_name for the brand 'Redmi' includes 'Redmi' at the beginning and clean the brand name without removing 'Redmi'.
    """
    model = model.lower()  # Normalize case
    
    # Check if 'redmi' is not at the beginning, add it if necessary
    if not model.startswith('redmi'):
        model = 'redmi ' + model
    
    # Replace '+' with 'plus'   
    model = re.sub(r'\+', ' plus', model)

    # Remove any extra descriptive information in parentheses or after commas
    model = re.sub(r'\(.*?\)', '', model).strip()
    model = re.sub(r',.*$', '', model).strip()
    model = re.sub(r'[\-\/].*$', '', model).strip()
    
    # Handle hyphens between numbers and specs
    model = re.sub(r'(\d+)-(\d+)', r'\1 \2', model)
    
    # Clean up unnecessary extra text but keep 'redmi'
    model = re.sub(r'\bredmi\b', 'redmi', model).strip()
    
    # Extract the core model name, keeping '4G' or '5G' if present
    model = re.sub(r'\b4g\b', '4g', model)  
    model = re.sub(r'\b5g\b', '5g', model)  

    return model.strip()  


# Function to apply based on brand
def process_model(row):
    brand = row['brand'].lower()
    model_name = row['model_name']
    product_title = row['product_title']
    
    if brand == 'samsung':
        return format_samsung_model(model_name)
    elif brand == 'xiaomi':
        return format_xiaomi_brand(model_name)
    elif brand == 'apple':
        return format_apple_model(model_name)   
    elif brand == 'honor':
        return format_honor_model(model_name)
    elif brand == 'oppo':
        return format_oppo_model(model_name)
    elif brand == 'nokia':
        return format_nokia_model(model_name,product_title)   
    elif brand == 'infinix':
        return format_infinix_model(model_name)
    elif brand == 'realme':
        return format_realme_model(model_name) 
    elif brand == 'redmi':
        return format_redmi_model(model_name)
    else:
        return model_name


# Apply function to the DataFrame
final_df['model_name'] = final_df.apply(process_model, axis=1)

##### Finally clean things up by removing products with too many missing values and ensuring we're only dealing with phones.

In [18]:
final_df = final_df.dropna(subset=["ram_gb", "storage", "screen_size_in", "resolution", "cpu_speed_ghz", "cpu_model"], how='all')
final_df.shape

(233, 27)

In [19]:
# Convert `date_column` to datetime
final_df['date_column'] = pd.to_datetime(final_df['date_column'], errors='coerce')

# Convert `price_usd` to numeric
final_df['price_egp'] = pd.to_numeric(final_df['price_egp'], errors='coerce')

In [20]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 233 entries, 0 to 253
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   date_column              233 non-null    datetime64[ns]
 1   site                     233 non-null    object        
 2   category                 233 non-null    object        
 3   brand                    233 non-null    object        
 4   model_name               233 non-null    object        
 5   product_title            233 non-null    object        
 6   price_egp                184 non-null    float64       
 7   os                       233 non-null    object        
 8   ram_gb                   217 non-null    object        
 9   storage                  230 non-null    object        
 10  screen_size_in           206 non-null    object        
 11  resolution               146 non-null    object        
 12  refresh_rate_hz          154 non-null    

In [21]:
final_df

,date_column,site,category,brand,model_name,product_title,price_egp,os,ram_gb,storage,...,wireless_carrier,cellular_technology,all_reviews,rate,discount,product_url,image_url,asin,batteries,model_number
0,2024-09-16,amazon_egy,mobile phones,samsung,s24 ultra,"samsung galaxy s24 ultra, ai phone, 256gb stor...",47900.0,android,12 gb,256 gb,...,cellular,NaN,"[{'reviewer': 'george', 'rating': '5.0 ', 'dat...",4.1,NaN,https://www.amazon.eg/-/en/Samsung-Storage-Tit...,https://m.media-amazon.com/images/I/719HV2e6-s...,b0cqz22q7l,NaN,sm-s928bzkcmea
1,2024-09-16,amazon_egy,mobile phones,apple,14 plus,new apple iphone 14 plus (128 gb) - starlight,34990.0,ios,NaN,128 gb,...,unlocked for all carriers,5g,"[{'reviewer': 'azizbek sharobidinov', 'rating'...",4.7,3,https://www.amazon.eg/-/en/New-Apple-iPhone-Pl...,https://m.media-amazon.com/images/I/51S9HOc4UL...,b0bdj59rrq,1 lithium polymer batteries required. (included),mq4y3aa/a
2,2024-09-16,amazon_egy,mobile phones,xiaomi,redmi note 13,"xiaomi redmi note 13 (mint green 8gb ram, 256 ...",8650.0,android 13.0,8 gb,256 gb,...,3,NaN,"[{'reviewer': 'sofianamr', 'rating': '5.0 ', '...",4.1,4,https://www.amazon.eg/-/en/Xiaomi-Redmi-Green-...,https://m.media-amazon.com/images/I/51Q+-U8vAV...,b0cqkddcvk,1 lithium polymer batteries required.,n7
4,2024-09-16,amazon_egy,mobile phones,honor,x7b,honor x7b 6-256 green,8690.0,android 13.0,6 gb,256 gb,...,china mobile,4g,[],NaN,NaN,https://www.amazon.eg/-/en/HONOR-6936520833290...,https://m.media-amazon.com/images/I/51ykLhmalQ...,b0cvhdld4p,NaN,6936520833290
6,2024-09-16,amazon_egy,mobile phones,xiaomi,redmi note 13 pro,"redmi note 13 pro (midnight black 12gb ram, 51...",15850.0,android 11.0,12 gb,512 gb,...,3,NaN,"[{'reviewer': 'carlos torres torres', 'rating'...",4.0,NaN,https://www.amazon.eg/-/en/Redmi-Note-Midnight...,https://m.media-amazon.com/images/I/51hrgXBsTa...,b0cqkdbvt8,1 lithium polymer batteries required. (included),n6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2024-09-16,amazon_egy,mobile phones,nokia,c10,"nokia c10 android smartphone, dual sim, 2gb ra...",2899.0,android 11.0,2 gb,32 gb,...,NaN,NaN,"[{'reviewer': 'مهندس .ممدوح فرج', 'rating': '5...",3.3,NaN,https://www.amazon.eg/-/en/Nokia-android-smart...,https://m.media-amazon.com/images/I/81LrJ-Q-Tg...,b0b39ln2n6,1 lithium ion batteries required. (included),NaN
250,2024-09-16,amazon_egy,mobile phones,realme,note 50,realme note 50 dual sim 64gb rom + 3gb ram (gs...,NaN,android,3 gb,64 gb,...,unlocked for all carriers,lte,"[{'reviewer': 'mohamed fathy', 'rating': '1.0 ...",3.3,NaN,https://www.amazon.eg/-/en/realme-Note-Factory...,https://m.media-amazon.com/images/I/71v1qIODQq...,b0d1gwyh9x,1 lithium ion batteries required. (included),rmx3834
251,2024-09-16,amazon_egy,mobile phones,samsung,s23,"samsung galaxy s23 ultra, 256gb, phantom black...",38450.0,android 12.0,12 gb,256 gb,...,cellular,NaN,"[{'reviewer': 'randa a.', 'rating': '5.0 ', 'd...",4.4,NaN,https://www.amazon.eg/-/en/Samsung-Galaxy-Phan...,https://m.media-amazon.com/images/I/71goZuIha-...,b0bt9fdz8n,1 lithium ion batteries required. (included),sm-s918b
252,2024-09-16,amazon_egy,mobile phones,oppo,a55,oppo a55 dual sim 4gb ram+128gb rom (6.5 inche...,6250.0,android 11.0,4 gb,128 gb,...,china mobile,NaN,"[{'reviewer': 'ايمان س.', 'rating': '5.0 ', 'd...",4.0,NaN,https://www.amazon.eg/-/en/OPPO-A55-Dual-128GB...,https://m.media-amazon.com/images/I/619NisKmiu...,NaN,NaN,NaN


In [22]:
# # Save DataFrame to CSV
# final_df.to_csv(f"E:\\Optimized-Amazon-project\\phones-cleaned-data\\{datetime.today().strftime('%Y-%m-%d')}.csv", index=False)